# Bar plot with confidence interval and samples
This notebook plots a bar graph from per-subject values extracted from REX (or another software) and plots the 90% confidence interval and all per-subject values as scatterplots.

If in addition you supply the ROIs nifti images (one per ROI), you will in addition get a nice glass brain image with all ROIs, and the bar plot will also include additional infos (ROI center coordinates + atlas regions names covered by ROI + same color of text as the glass brain).

Version 1.7.0

In [ ]:
%load_ext autoreload
%autoreload 2
# BEWARE: autoreload works on functions and on general code, but NOT on new class methods:
# if you add or change the name of a method, you have to reload the kernel!
# also it will fail if you use super() calls in the classes you change
# ALSO AUTORELOAD SHOULD BE THE FIRST LINE EVER EXECUTED IN YOUR IPYTHON NOTEBOOK!!!

# Profilers:
# http://pynash.org/2013/03/06/timing-and-profiling/
# http://mortada.net/easily-profile-python-code-in-jupyter.html
# use %lprun -m module func(*args, **kwargs)
try:
    %load_ext line_profiler
    %load_ext memory_profiler
except ImportError as exc:
    pass

In [ ]:
# Generate figure inside IPython Notebook (must be called before any import of matplotlib, direct or indirect!)
%matplotlib inline

import math
import matplotlib.pyplot as plt
import matplotlib.colors as pltcol
import numpy as np
import nibabel as nib
import pandas as pd
import textwrap
from nilearn import image
from nilearn import plotting
try:
    from adjustText import adjust_text  # you will need this lib if you want to plot subjects ids without overlap: pip install adjustText
except ImportError as exc:  # else we will just plot the subjects labels without any adjustment (so the position can overlap with the points and with each others)
    adjust_text = None
    pass

In [ ]:
# PARAMETERS - EDIT ME
groups = [9, 9]  # set the number of items/values for each group
groups_labels = ['S2', 'W1']  # set names for each group
groups_order = range(len(groups))  # in which order we plot each bar. Leave as is to plot bars in the original order, or provide list of group nb to shuffle as you want (this is only a display parameter, it does not change the results)
#subjects_labels = range(1, sum(groups)+1)  # set ID for each subject (can be any string, all that matters is that it is the same order as the input values: first value here is the first value in imported rex data file, etc)
#subjects_labels = range(1, groups[0]+1)*len(groups)  # use this for within-subject analyses, where you want the labels to be the same range in both bars
show_subjects_labels = True  # show label for each subject's point?
show_subjects_paired = True  # paired/longitudinal analysis, both groups contain in fact the same subjects but in different conditions, enable this option to restart counting subjects ids to 1 for each condition
rex_data_filepath = ['testconjunc.cluster00%i.rex.data.txt' % i for i in xrange(1,6)]
rex_maps_filepath = ['testconjunc.cluster00%i.rex.roi.img' % i for i in xrange(1,6)]  # specify here the filenames, in the same order as the input text files, to plot the ROIs on a brain using nilearn
atlas_cmap = plt.cm.Set1  # colormap to use for ROIs (only if map provided)
show_colorbar = False  # display the colorbar on the brain image?
dpi_resolution = 300  # dpi resolution when saving on-disk

In [ ]:
def load_maps(list_imgs, voxel_threshold=None):
    if voxel_threshold is None:
        voxel_threshold = 0.0001 # minimum threshold to consider as a voxel and not just background noise (because background voxels can be 0.000001 for example), can be float or str ('1%' to give a percentage). TODO: autodetect minimum value (can be -4, 0.02, etc) as the background and use it as the threshold value.
    # Load masks and resample to first
    imgs = []
    for img in list_imgs:
        im = image.load_img(img)
        if imgs:
            if im.shape != imgs[0].shape:
                im = image.resample_to_img(im, imgs[0])
        im = image.threshold_img(im, voxel_threshold)
        imgs.append(im)
    return imgs

In [ ]:
# SANITY CHECKS
#if show_subjects_labels:
    #if len(subjects_labels) != sum(groups):
        #raise(ValueError('subjects_labels does not contain the same number of subjects as groups!'))

In [ ]:
# Check filepaths are lists and not just string
if isinstance(rex_data_filepath, str):
    rex_data_filepath = [rex_data_filepath]
if isinstance(rex_maps_filepath, str):
    rex_maps_filepath = [rex_maps_filepath]
# Loading data from Rex csv
dfraw = []
nb_rois = len(rex_data_filepath)
for fpath in rex_data_filepath:
    dfraw.append(pd.read_csv(fpath, index_col=False, header=None, squeeze=True))
    if len(dfraw[-1]) != sum(groups):
        raise(ValueError('the number of values in the provided txt file is not the same as the supplied groups count (ie, you did not specify the correct number of subjects!): %i vs %i, please check your parameters!' % (sum(groups), len(dfraw[-1]))))
    print(dfraw[-1])

In [ ]:
# Extract the values for each group in a separate dataframe
df_g = []
for i in xrange(nb_rois):
    start = 0
    for g in groups:
        # Extract current subset of values for the current number of subjects (g is the number of subjects in current group)
        df_g.append(dfraw[i][start:start+g])
        # Next start after where we left here (to extract next group's values)
        start = start+g
        if show_subjects_paired:
            # Reset indices to restart at 0 the subjects counting because it's the same subjects (longitudinal analysis)
            df_g[-1].index = list(range(0,len(df_g[-1])))
        #else:
            # Continue the numbering (different groups of subjects)

df_g

In [ ]:
# Helper functions
import numpy as np
import scipy.stats

def comp_ci(a):
    '''Calculates the 90% confidence interval from a vector.
    From the excellent SO answer by Ulrich Stern: https://stackoverflow.com/a/34474255/1121352'''
    return scipy.stats.t.interval(0.90, len(a)-1, loc=np.mean(a), scale=scipy.stats.sem(a))

In [ ]:
def find_cluster_center(im, mricron=False):
    from nilearn.image.resampling import reorder_img, coord_transform
    # Project coordinates to reduced space
    if mricron:
        im2 = reorder_img(im, resample='continuous')  # DO NOT USE: this will convert to MRIcron coordinates space (ie, [0, 100]) but it will mess things up for nilearn!
        # Get indices of nonzero values
        matches = im2.get_data().nonzero()
    else:
        # Get indices of nonzero values
        matches = im.get_data().nonzero()
    # Compute the euclidian middle of the cluster, from the nonzero values indices (= coordinates)
    center = np.mean(matches, axis=1)
    # Project center coordinates to brain space (ie, instead of [0, 100] range, it will be [-50, 50] -- I picked these numbers randomly, you see the idea)
    if not mricron:
        center = coord_transform(center[0], center[1], center[2], im.affine)  # disable this to get MRIcron space
    return center

In [ ]:
# Plot ROIs on glass brain images!
if rex_maps_filepath:
    imgs = load_maps(rex_maps_filepath)

    #plotting.plot_prob_atlas(imgs, view_type="filled_contours",
    #                    title="lala", colorbar=True, cut_coords=(0,0,0), draw_cross=True, cmap=pltcol.ListedColormap(['b', 'g', 'r', 'c', 'm'], name='from_list', N=None))
    #plotting.plot_roi(imgs[0])

    centers = []
    fig = plotting.plot_glass_brain(None, cmap=atlas_cmap, alpha=0.5)  # initialize the glass brain images
    for c, im in enumerate(imgs):
        # For each ROI
        # Get the center (to plot the marker)
        center = find_cluster_center(im)
        centers.append(center)
        print('Center found at: ' + str(center))
        # Assign a unique value to this cluster's voxels (to get a different color)
        imdata = im.get_data()  # Convert to numpy structure
        imdata[imdata != 0] = c+1  # Assign unique value
        im2 = nib.Nifti1Image(imdata, affine=im.affine)  # convert back to a nifti file in-memory to supply to nilearn
        # Show colorbar?
        cbar = False
        if show_colorbar and c == (nb_rois-1):
            # Can only plot the colorbar at the last iteration, else nilearn will spit an error (cannot use multiple colorbars)
            cbar = True
        # Plot the clusters
        fig.add_overlay(im2, vmin=1, vmax=nb_rois, cmap=atlas_cmap, colorbar=cbar)
        # Plot the markers (clusters' centers)
        fig.add_markers([center], marker_color=['k'], marker_size=20)

# Save figure
fig.savefig('rois_glass_brain.png', dpi=dpi_resolution)
print('Image saved in rois_glass_brain.png')

In [ ]:
from nilearn import plotting, datasets

def get_atlas_label(atlas, region_idx):
    """
    Get atlas label for one specific region index
    """
    return atlas['labels'][atlas['indices'].index(str(region_idx))]

def get_atlas_labels(imgs, atlas_choice='aal2', verbose=False):
    """
    Get the list of atlas regions covered by clusters, from a list of nifti maps loaded in-memory via nibabel
    atlas_choice is optional, can be 'aal2' or 'SPM12'
    """
    voxel_threshold = 0.0001 # minimum threshold to consider as a voxel and not just background noise (because background voxels can be 0.000001 for example), can be float or str ('1%' to give a percentage). TODO: autodetect minimum value (can be -4, 0.02, etc) as the background and use it as the threshold value.
    # Atlas
    atlas_choice = 'aal2' # anatomytoolbox or aal2
    if atlas_choice == 'anatomytoolbox':
        atlas_path = 'masks\AnatomyToolbox_Atlas_Map.nii'  # TODO: build atlas variable with all infos and data (labels, indices, nib niftiimage with affine etc)
    else:
        atlas = datasets.fetch_atlas_aal(version='SPM12', data_dir='atlas')

    # Show some infos about atlas
    atlas_im = image.load_img(atlas.maps)
    if verbose:
        print('Atlas shape: %s' % str(atlas_im.shape))
    nb_regions = len(np.unique(atlas_im.get_data()))- 1
    if verbose:
        print('%i regions in this atlas: %s' % (nb_regions, str(np.unique(atlas_im.get_data())))) # 48 regions because 0 is background
        print('%i labels' % len(atlas['labels']))
        print('%i indices: %s' % (len(atlas['indices']), atlas['indices']))
        print(atlas.keys())

    # Resample masks to atlas size
    imgs2 = []
    for img in imgs:
        if img.shape != atlas_im.shape:
            img = image.resample_to_img(img, atlas_im)
        img = image.threshold_img(img, voxel_threshold)
        imgs2.append(img)
    imgs = imgs2
    del imgs2
    imgs[0].shape

    # Extract activated atlas brain regions for each mask
    maps_regions = []
    maps_regions_idxs = []
    maps_regions_count = []
    for img in imgs:
        # Extract only non zeros voxels indices from mask
        im_data = img.get_data()
        #np.extract(im_data>0, im_data)
        vox_thres = np.nonzero(im_data)
        # Compare with atlas regions to extract region indices
        atlas_data = atlas_im.get_data()
        region_indices = set()
        region_count = {}
        for x in zip(*vox_thres): # walk through all non zero voxels of mask
            region_idx = atlas_data[x] # get equivalent voxel from atlas
            if region_idx != 0: # if not background
                # Append region index into the set (so that they are unique)
                region_indices.add(region_idx)
                # Increase the count of voxels activated in this region
                region_label = get_atlas_label(atlas, region_idx)
                if region_label not in region_count:
                    region_count[region_label] = 0
                region_count[region_label] += 1
        if 0 in region_indices:
            region_indices.remove(0) # remove background, not part of the atlas labels
        if verbose:
            print('Atlas indices of brain regions activated in current mask: %s' % str(sorted(region_indices)))
        # Extract brain region names from atlas that are present in this mask
        matching_idxs = [int(idx) in region_indices for idx in atlas['indices']]
        map_brain_regions = filter(None, [label if match else None for label, match in zip(atlas['labels'], matching_idxs)])
        maps_regions.append(map_brain_regions)
        maps_regions_idxs.append(region_indices)
        maps_regions_count.append(region_count)

    return maps_regions, maps_regions_idxs, maps_regions_count

if rex_maps_filepath:
    maps_regions, _, _ = get_atlas_labels(imgs)
    print('Found the following regions covered by the ROIs clusters:')
    for i in xrange(nb_rois):
        print('ROI %i: %s' % (i, ', '.join(maps_regions[i])))

In [ ]:
# Plot!

# Plotting parameters
ylim = None  # limit y axis to these values. Set to None to use default limits automatically detected by matplotlib.
width = 0.6  # width of the bars - do not change, it is an internal parameter and does not impact the visualization
margins = [0.8, 0.64] # margin we will set and account for to consistently scale the bars whatever the figsize is, in inches
figsize = [width*len(groups)*nb_rois+sum(margins), 3]  # figure size, in inches, set to None to use default (don't forget to add +width because the bars are in the middle of tickes, thus to have room you must account for +1 ghost bar on the outer edges of the fig)
colors = ['b', 'r', 'm', 'y', 'c', 'g', 'k']
bars_color_by_roi = True  # group bars colors by ROI, or differenciate by group/bar? NOTE: can only work if a map is provided
bars_alpha = 0.5  # bars coloring transparency, allows to more easily see the subjects points on top
ylabel = 'Effect sizes'
roi_labels_yoffset = -0.17  # how much shift to move the roi labels downward?
roi_labels_maxlen = 8  # this is the maximum length (nb of characters) per tick/group
roi_labels_fontsize = 11  # roi labels fontsize
plot_params = {'legend.fontsize': 15,  # various plot parameters
         #'figure.figsize': figsize,
         'axes.labelsize': 20,
         'axes.titlesize':30,
         'xtick.labelsize':15,
         'ytick.labelsize':15}
subjects_fontsize = 12  # fontsize for the labels per subjects points
subjects_labels_alpha = 0.5  # how much transparency for the subjects labels
boxplot_enable = False  # use a boxplot or a barplot? True for boxplot, False for barplot
boxplot_filled = True  # fill boxplot with color or only the edges?

# Prepare the ROI colorbar range normalization
if rex_maps_filepath:
    norm = pltcol.Normalize(vmin=1,vmax=nb_rois)  # need to normalize the values we will input to the colormap to be onpoint with nilearn
# Plotting each bar
plt.rcParams.update(plot_params)  # update the plot parameters right away, else they won't be updated until next cell execution
ticks = np.arange(width, width+(width*len(groups)*nb_rois), width)  # do not modify this
fig, ax = plt.subplots()
if figsize:
    fig.set_size_inches(figsize[0], figsize[1], forward=True)
for roi_id in xrange(nb_rois):
    last_j = 0
    texts = []
    for i, gi in enumerate(groups_order):
        # Get the data for the selected group
        dg = df_g[len(groups)*roi_id + gi]
        # Draw bars with error bar
        bar_color = colors[i] if not bars_color_by_roi or not rex_maps_filepath else atlas_cmap(norm(roi_id+1))
        xpos = ticks[len(groups)*roi_id+i]+(float(width)/2)
        if boxplot_enable:
            # Boxplot
            bp = ax.boxplot(dg, positions=[xpos], patch_artist=boxplot_filled, notch=True, vert=True, whis=[10, 90], widths=width*0.9, showmeans=True, showfliers=True, sym='_', flierprops={'markeredgecolor': bar_color, 'markeredgewidth': 2})
            ax.axhline(y=0, color='grey', linestyle='dotted')  # add a horizontal line at 0 to separate positive from negative effect size
            # Change color of outline and fill
            bp['boxes'][0].set(color=bar_color)
            if boxplot_filled:
                bp['boxes'][0].set_facecolor(bar_color)
                bp['boxes'][0].set_alpha(bars_alpha)
            # Change color of whiskers and caps
            for whisk in bp['whiskers']:
                whisk.set(color='black', linewidth=1)
            for cap in bp['caps']:
                cap.set(color='black', linewidth=1)
            # Median bar
            bp['medians'][0].set(color='black')
            # Mean point
            bp['means'][0].set(color='black', markerfacecolor='black', markeredgecolor='black')
        else:
            # Barplot
            ax.bar(xpos, dg.mean(), width=width, yerr=(dg.mean() - comp_ci(dg)[1]), alpha=bars_alpha, color=bar_color, error_kw={'ecolor': 'k', 'elinewidth': 1, 'capsize': 15, 'capthick': 1, 'barsabove': False})
        # Add scatter points for each subject
        scatter_x = ticks[len(groups)*roi_id+i]+(float(width)/2)
        if not boxplot_enable:
            # If barplot, plot all subjects points, else if boxplot then the boxplot will take care of plotting the outliers only
            ax.scatter([scatter_x] * len(dg), dg, color=bar_color, marker='_', s=30)  # multiply scatter_x to repeat x position for as many values as we have in dg
        # Add label for each subject scatter point
        if show_subjects_labels:
            for j, y in enumerate(dg):
                text = dg.index.values[j] + 1
                # Add label for all subjects if barplot, else only for outliers if boxplot
                if not boxplot_enable or y in bp['fliers'][0].get_ydata():
                    t = ax.text(scatter_x, y, text, alpha=subjects_labels_alpha, fontsize=subjects_fontsize)
                    texts.append(t)
            last_j += j+1
    # Adjust label for each subject text placement to avoid overlapping
    if show_subjects_labels and adjust_text is not None:
        #texts = subjects_labels[last_j:end]
        adjust_text(texts,
                    text_from_points=True,
            only_move={'text':'xy', 'objects':'x'}, force_text=0.05, force_objects=1.0) #, arrowprops=dict(arrowstyle="->", color='r', lw=0.5))

# Change the ticks to set the groups names (and place the labels nicely)
ax.set_xticks([t + float(width)/2 for t in ticks])  # place in the middle of each bar (position tick t + half of bar width)
groups_labels_reordered = [groups_labels[x] for x in groups_order]  # reorder the groups labels
ax.set_xticklabels(groups_labels_reordered * nb_rois)
# Add extra x-axis ROI labels: ROIs centers, colors and names if available (ie, if maps are provided)
if rex_maps_filepath:
    for ri in xrange(nb_rois):
        tick_roi_offset = ri*len(groups)  # plus offset by the number of past ROIs
        middletick = math.floor(len(groups)*0.5)  # middle tick of first ROI = half of nb of groups
        tick_even_offset = 0.5 if len(groups) % 2 == 1 else 0  # offset to be in the middle of two ticks if the number of ticks/groups is even
        axis_scaling = (1.0/len(ticks))  # scale to range [0.0, 1.0], because we use transform ax.transAxes (so we are in the space of the axis, not ticks)
        t = ax.text((tick_roi_offset+middletick+tick_even_offset)*axis_scaling, roi_labels_yoffset, '%i,%i,%i\n%s' % (round(centers[ri][0]), round(centers[ri][1]), round(centers[ri][2]), '\n'.join(textwrap.wrap(', '.join(maps_regions[ri]), width=roi_labels_maxlen*len(groups), break_long_words=True))),
                rotation=0, verticalalignment='top', horizontalalignment='center',
                transform=ax.transAxes,
                color=atlas_cmap(norm(ri+1)),
                fontsize=roi_labels_fontsize)
        # pltcol.rgb2hex(atlas_cmap(norm(ri+1)))  # to get the hex value of the color
# Force draw the plot (with tight layout)
plt.tight_layout()  # MUST BE DONE BEFORE subplots_adjust!
# Ensure proper and consistent scaling by setting the margins
fig.subplots_adjust(left=margins[0]/figsize[0], right=1-margins[1]/figsize[0])  # Courtesy of ImportanceOfBeingErnest: https://stackoverflow.com/a/48532569/1121352
if ylim:
    ax.set_ylim(ylim)
ax.set_xlim([ticks[0], ticks[-1]+width])
plt.ylabel(ylabel)
plt.show()


# Save the figure
fig.savefig('rois_bars.png', dpi=dpi_resolution, bbox_inches='tight')
print('Image saved in rois_bars.png')

-----------------------------------------------------------

In [ ]:
# TEST: inconsistent scaling, bar width change with number of bars
nb_bars_list = [2, 10]

for i, nb_bars in enumerate(nb_bars_list):
    figsize = [1+nb_bars, 5]
    points = [np.random.randn(10) for x in xrange(nb_bars)]
    fig, ax = plt.subplots()
    if figsize:
        fig.set_size_inches(figsize[0], figsize[1], forward=True)
    ticks = np.arange(1, 1+nb_bars, 1)
    for b in xrange(nb_bars):
        ax.bar(ticks[b], points[b].mean(), 1)
    plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
    fig.savefig('test%i' % i, bbox_inches='tight')

In [ ]:
# TEST: consistent scaling with same bar size whatever is the number of bars
# Courtesy of ImportanceOfBeingErnest: https://stackoverflow.com/a/48532569/1121352
import matplotlib.pyplot as plt
import numpy as np

nb_bars_list = [2, 10]
margleft = 0.8 # inch
margright= 0.64 # inch
barwidth = 1 # inch


for i, nb_bars in enumerate(nb_bars_list):
    # Resize proportionally to the number of bars
    axwidth = nb_bars*barwidth # inch
    figsize = [margleft+axwidth+margright, 5]
    # Prepare the ticks
    ticks = np.arange(1, 1+nb_bars, 1)
    # Generate random points
    points = [np.random.randn(10) for x in xrange(nb_bars)]
    # Make the plot
    fig, ax = plt.subplots(figsize=figsize)
    fig.subplots_adjust(left=margleft/figsize[0], right=1-margright/figsize[0])
    for b in xrange(nb_bars):
        ax.bar(ticks[b], points[b].mean())
    ax.set_xlim(ticks[0]-0.5,ticks[-1]+0.5)
    fig.savefig('test%i' % i, bbox_inches='tight')
plt.show()